In [1]:
PROMPT_ID = 'financial_health_v1'
LABEL_KEY = 'employment_status'

from helpers.loaders import load_config
import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import pandas as pd
import plotly

device = torch.device('cuda')
label_encoding_map, test_examples = load_config('label_maps.yaml', prompt_id = PROMPT_ID, label_key = LABEL_KEY) # Get config

## Get Base Model

In [2]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification
# from transformers import RobertaTokenizer, RobertaForSequenceClassification

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels = len(label_encoding_map)).to(device)

# Num params
sum(p.numel() for p in model.parameters())

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


11685891

## Get Data & Configs

In [3]:
# Get data
from helpers.db import get_postgres_query

raw_data = get_postgres_query(
    f"""
    WITH t0 AS (
        SELECT 
            a.post_id, a.label_value AS value, a.label_rationale AS rationale,
            CONCAT(TRIM(b.title), '\n', REGEXP_REPLACE(TRIM(b.selftext), '[\t\n\r]', ' ', 'g')) AS input_text
        FROM text_scraper_reddit_llm_scores_v2 a
        INNER JOIN text_scraper_reddit_scrapes b
            ON a.scrape_id = b.scrape_id
        WHERE 
            a.PROMPT_ID = '{PROMPT_ID}' 
            AND a.label_key = '{LABEL_KEY}'
    )
    -- Select where expected token count <= 512
    SELECT * 
    FROM t0
    WHERE ARRAY_LENGTH(REGEXP_SPLIT_TO_ARRAY(input_text, '\\s+'), 1) * 1.4 <= 512
    """
    ).merge(label_encoding_map, how = 'inner', on = 'value')

# 20k total, 16k train. Overweight low-frequency label to the maximum extent possible.
input_data =\
    raw_data\
    .sample(n = int(8000 * 1/.8), weights = 1/raw_data.groupby('value')['value'].transform('count')) 

In [4]:
input_data.groupby('label_encode').agg(count = ('post_id', 'count'))

,count
label_encode,
0,2680
1,3844
2,3476


In [5]:
# Prep Datasets & Dataloader
from helpers.loaders import TextDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
B = 16

train_df, test_df = train_test_split(input_data, test_size = .2, random_state = 1234)
train_ds, test_ds = [TextDataset(tokenizer, x['input_text'].tolist(), x['label_encode'].tolist()) for x in [train_df, test_df]]
train_dl, test_dl = [DataLoader(x, batch_size = B) for x in [train_ds, test_ds]]

examples_ds = TextDataset(tokenizer, [x['text'] for x in test_examples], [x['label'] for x in test_examples])
test_df.head(5)

,post_id,value,rationale,input_text,label_encode
23932,t3_14x6gqp,employed,The user mentions considering quitting their j...,Should I quit my job because of a family emerg...,1
30670,t3_x80dpp,unemployed,The user was fired and is seeking a new job.,I was FIRED! Then two years later was forced t...,0
17146,8xhd98,employed,"The user mentions 'the company I work for,' in...",Welcoming suggestions about financing Graduate...,1
32968,t3_icvy5v,unemployed,The user was terminated from their previous jo...,Advice Needed - Terminated for Embarrassing Re...,0
34255,t3_zl10bv,unemployed,The user mentions struggling with freelancing ...,Have you ever felt a lack of confidence in you...,0


## Test Initial Eval

In [6]:
from helpers.loaders import eval_performance, eval_performance_as_str

print(eval_performance(model, test_ds, device, batch_size = 16, verbose = True))
print(eval_performance_as_str(model, examples_ds, device))

125it [00:37,  3.35it/s]

{'mean_nll': 1.1455338, 'accuracy': 0.328, 'count': 2000}
Correct: 4/9
✅ [0.4] - Started searching for jobs and struggling to find anything
❌ [0.31] - Just received a new job offer with a big pay raise!
❌ [0.27] - How to find a good budgeting app?
✅ [0.4] - Emotionally exhausted from this BS market
I just need to rant. I never cried so much within a week until now. I have bills to pay, rent is over 2k for a studio, there are no jobs here. I am so tired because this job market is exhausting and everyone is saying its fine.
❌ [0.28] - Coworker got a bigger raise than me.
❌ [0.26] - Advice on choosing a career path.
✅ [0.39] - As a recent college graduate, I thought it wouldn't be that hard to find a job that paid $20 an hour that but I was wrong. There are no entry level jobs in my field or any job that could potentially get my foot in the door pay so little. What is the point of having a college degree if full time jobs won't even allow me to move out and rent a room.
✅ [0.45] - What wo

## Train

In [7]:
# Set train confs
conf = {
    'epochs': 5,
    'optim_lr': 2e-5,
    'sched_gamma': .5,
    'sched_steps': 400, # Drops every 400 * 16 iters
    'clip': False
}

optimizer = torch.optim.Adam(model.parameters(), lr = conf['optim_lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = conf['sched_steps'], gamma = conf['sched_gamma'])                                                                                                  

In [8]:
# Create save dir
from helpers.storage import create_save_dir
import json

save_dir = create_save_dir(prompt_id = PROMPT_ID, label_key = LABEL_KEY)
with open(f'{save_dir}/conf.json', "w") as outfile:
    outfile.write(json.dumps(conf, indent = 4))

print([x/1e9 for x in torch.cuda.mem_get_info()])
torch.cuda.empty_cache()
[x/1e9 for x in torch.cuda.mem_get_info()]

[5.930745856, 8.589410304]


[6.555697152, 8.589410304]

In [9]:
# Known to work well: 3e-5 with no clipping; 2e-5 with clipping, gamma=.5
train_losses = []
test_losses = []

f = plotly.graph_objects.FigureWidget().add_scatter(name = 'train', x = [], y = []).add_scatter(name = 'test', x = [], y = [])
for e in range(conf['epochs']): f.add_vline(x = e * len(train_dl))
display(f)

for epoch in range(conf['epochs']):
    print(f'***** Epoch {epoch + 1} ')
    torch.cuda.empty_cache() 
    model.train()
    epoch_loss = 0
    
    for step, batch in tqdm(enumerate(train_dl)):
        
        optimizer.zero_grad()
        outputs = model(batch['input_ids'].to(device), batch['attention_mask'].to(device))
        loss = F.cross_entropy(outputs['logits'], batch['labels'].to(device))
    
        loss.backward()
        
        if conf['clip']:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
        optimizer.step()
        scheduler.step()

        # Logging and eval
        epoch_loss = epoch_loss + loss.item()
        train_losses.append({'epoch': epoch, 'step': step, 'loss': loss.item()})
        if step % 50 == 0 and (step > 0 or epoch > 0): 
            test_eval_res = eval_performance(model, test_ds, device, batch_size = 16, verbose = False)
            test_losses.append({'epoch': epoch, 'step': step, 'loss': test_eval_res['mean_nll']})
            print(
                f"Step {step}/{len(train_dl)} | " +
                f"Train Loss: {np.mean([t['loss'] for t in train_losses][-50:])} | Test Loss: {test_losses[-1]['loss']} | " +
                f"LR: {optimizer.param_groups[0]['lr']} | RAM: {round(torch.cuda.memory_allocated(0)/1024**3,1)} GB"
            )
            print(eval_performance_as_str(model, examples_ds, device))
            # Update graph
            losses_df = pd.concat([
                pd.DataFrame(train_losses).assign(type = 'train', loss = lambda df: df['loss'].rolling(window = 50).mean()).dropna(axis = 0),
                pd.DataFrame(test_losses).assign(type = 'test')
            ]).assign(i = lambda df: (df['epoch'] * len(train_dl)) + df['step'] + 1)     
            f.data[0].x, f.data[1].x = [losses_df[losses_df['type'] == j]['i'].tolist() for j in ['train', 'test']]
            f.data[0].y, f.data[1].y = [losses_df[losses_df['type'] == j]['loss'].tolist() for j in ['train', 'test']]

        # Save ts and ckpt 
        if step % 100 == 0 and (step > 0 or epoch > 0):
            torch.save({
                'epoch': epoch, 'step': step, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict(),
                'train_losses': train_losses, 'test_losses': test_losses
            }, f"{save_dir}/epoch_{str(epoch).rjust(3, '0')}_step_{str(step).rjust(5, '0')}.ckpt")
            
            model.eval()
            ex = next(iter(test_dl))
            model_scripted = torch.jit.trace(model, (ex['input_ids'].to(device), ex['attention_mask'].to(device)), strict = False)
            model_scripted.save(f"{save_dir}/epoch_{str(epoch).rjust(3, '0')}_step_{str(step).rjust(5, '0')}.pt")
            model.train()

FigureWidget({
    'data': [{'name': 'train', 'type': 'scatter', 'uid': 'bde544a0-4216-457c-add8-b86ad79f9ba9', 'x': [], 'y': []},
             {'name': 'test', 'type': 'scatter', 'uid': 'b33ff2ca-4869-4546-a925-192df81b555a', 'x': [], 'y': []}],
    'layout': {'shapes': [{'type': 'line', 'x0': 0, 'x1': 0, 'xref': 'x', 'y0': 0, 'y1': 1, 'yref': 'y domain'},
                          {'type': 'line', 'x0': 500, 'x1': 500, 'xref': 'x', 'y0': 0, 'y1': 1, 'yref': 'y domain'},
                          {'type': 'line', 'x0': 1000, 'x1': 1000, 'xref': 'x', 'y0': 0, 'y1': 1, 'yref': 'y domain'},
                          {'type': 'line', 'x0': 1500, 'x1': 1500, 'xref': 'x', 'y0': 0, 'y1': 1, 'yref': 'y domain'},
                          {'type': 'line', 'x0': 2000, 'x1': 2000, 'xref': 'x', 'y0': 0, 'y1': 1, 'yref': 'y domain'}],
               'template': '...'}
})

***** Epoch 1 


50it [06:01,  6.92s/it]

Step 50/500 | Train Loss: 1.091649091243744 | Test Loss: 1.0202927589416504 | LR: 2e-05 | RAM: 0.2 GB


51it [06:44, 17.67s/it]

...

Correct: 9/9
✅ [0.88] - Started searching for jobs and struggling to find anything
✅ [0.77] - Just received a new job offer with a big pay raise!
✅ [0.87] - How to find a good budgeting app?
✅ [0.89] - Emotionally exhausted from this BS market
I just need to rant. I never cried so much within a week until now. I have bills to pay, rent is over 2k for a studio, there are no jobs here. I am so tired because this job market is exhausting and everyone is saying its fine.
✅ [0.79] - Coworker got a bigger raise than me.
✅ [0.83] - Advice on choosing a career path.
✅ [0.73] - As a recent college graduate, I thought it wouldn't be that hard to find a job that paid $20 an hour that but I was wrong. There are no entry level jobs in my field or any job that could potentially get my foot in the door pay so little. What is the point of having a college degree if full time jobs won't even allow me to move out and rent a room.
✅ [0.49] - What would be considered a decent salary for an apartment costi

400it [1:03:52,  8.40s/it]

Step 400/500 | Train Loss: 0.5237972721457481 | Test Loss: 0.7496311664581299 | LR: 2.5e-06 | RAM: 0.2 GB
Correct: 9/9
✅ [0.81] - Started searching for jobs and struggling to find anything
✅ [0.66] - Just received a new job offer with a big pay raise!
✅ [0.88] - How to find a good budgeting app?
✅ [0.89] - Emotionally exhausted from this BS market
I just need to rant. I never cried so much within a week until now. I have bills to pay, rent is over 2k for a studio, there are no jobs here. I am so tired because this job market is exhausting and everyone is saying its fine.
✅ [0.74] - Coworker got a bigger raise than me.
✅ [0.85] - Advice on choosing a career path.
✅ [0.6] - As a recent college graduate, I thought it wouldn't be that hard to find a job that paid $20 an hour that but I was wrong. There are no entry level jobs in my field or any job that could potentially get my foot in the door pay so little. What is the point of having a college degree if full time jobs won't even allow m

418it [1:07:18,  9.66s/it]


KeyboardInterrupt: 

In [ ]:
# from peft import LoraConfig, TaskType, get_peft_model

# lora_config = LoraConfig(
#     task_type = TaskType.SEQ_CLS, r = 4, lora_alpha = 1, lora_dropout = 0.1
# )

# model = get_peft_model(model, lora_config)
# model.print_trainable_parameters()
# print(model.roberta.encoder.layer[0].attention.self)